In [19]:
import pandas as pd
import numpy as np

# Date-Based Features
*  Extract year, month, day, dayofweek, is_weekend


In [20]:
def add_date_features(df: pd.DataFrame) -> pd.DataFrame:
    df['Year'] = df['Date'].dt.year
    df['Month'] = df['Date'].dt.month
    df['Day'] = df['Date'].dt.day
    df['DayOfWeek'] = df['Date'].dt.dayofweek
    df['IsWeekend'] = df['DayOfWeek'].isin([5, 6]).astype(int)
    return df

# Lag Features
*  Previous day prices: lagged Close, Volume, etc.

In [21]:
def add_lag_features(df: pd.DataFrame, columns: list, lags: list = [1, 2, 3]) -> pd.DataFrame:
    for col in columns:
        for lag in lags:
            df[f'{col}_lag_{lag}'] = df[col].shift(lag)
    return df

# Rolling Statistics
*  Rolling mean/median for 7, 14, 30 days (on `Close`, `Volume`)
*  Rolling std dev, rolling max/min

In [22]:
def add_rolling_features(df: pd.DataFrame, columns: list, windows: list = [7, 14, 30]) -> pd.DataFrame:
    for col in columns:
        for win in windows:
            df[f'{col}_roll_mean_{win}'] = df[col].rolling(window=win).mean()
            df[f'{col}_roll_median_{win}'] = df[col].rolling(window=win).median()
            df[f'{col}_roll_std_{win}'] = df[col].rolling(window=win).std()
            df[f'{col}_roll_max_{win}'] = df[col].rolling(window=win).max()
            df[f'{col}_roll_min_{win}'] = df[col].rolling(window=win).min()
    return df

# Volatility
*  Percentage change
*  Daily return
*  Rolling return

In [23]:
def add_volatility_features(df: pd.DataFrame) -> pd.DataFrame:
    df['Pct_Change'] = df['Close/Last'].pct_change()
    df['Daily_Return'] = df['Close/Last'].diff()
    df['Rolling_Return_7'] = df['Daily_Return'].rolling(7).mean()
    df['Rolling_Return_30'] = df['Daily_Return'].rolling(30).mean()
    return df

# Technical Indicators
*  Moving Averages (SMA, EMA)
*  RSI (Relative Strength Index)
*  MACD
*  Bollinger Bands

In [24]:
def add_technical_indicators(df: pd.DataFrame) -> pd.DataFrame:
    # SMA / EMA
    df['SMA_10'] = df['Close/Last'].rolling(window=10).mean()
    df['EMA_10'] = df['Close/Last'].ewm(span=10, adjust=False).mean()

    # RSI
    delta = df['Close/Last'].diff()
    gain = np.where(delta > 0, delta, 0)
    loss = np.where(delta < 0, -delta, 0)
    avg_gain = pd.Series(gain).rolling(window=14).mean()
    avg_loss = pd.Series(loss).rolling(window=14).mean()
    rs = avg_gain / (avg_loss + 1e-10)
    df['RSI_14'] = 100 - (100 / (1 + rs))

    # MACD
    ema_12 = df['Close/Last'].ewm(span=12, adjust=False).mean()
    ema_26 = df['Close/Last'].ewm(span=26, adjust=False).mean()
    df['MACD'] = ema_12 - ema_26
    df['MACD_Signal'] = df['MACD'].ewm(span=9, adjust=False).mean()

    # Bollinger Bands
    rolling_mean = df['Close/Last'].rolling(window=20).mean()
    rolling_std = df['Close/Last'].rolling(window=20).std()
    df['Bollinger_Upper'] = rolling_mean + 2 * rolling_std
    df['Bollinger_Lower'] = rolling_mean - 2 * rolling_std

    return df

# Target Variable
*  Optionally create a target for prediction (e.g., Next_Day_Close, or classification label Up/Down)

In [25]:
def add_target_variable(df: pd.DataFrame) -> pd.DataFrame:
    df['Target_Close_Next_Day'] = df['Close/Last'].shift(-1)
    df['Target_UpDown'] = (df['Target_Close_Next_Day'] > df['Close/Last']).astype(int)
    return df

In [26]:
def engineer_features(file_path: str, save_path: str) -> pd.DataFrame:
    df = pd.read_csv(file_path, parse_dates=['Date'])

    df = df.sort_values('Date').reset_index(drop=True)
    df = add_date_features(df)
    df = add_lag_features(df, columns=['Close/Last', 'Volume'])
    df = add_rolling_features(df, columns=['Close/Last', 'Volume'])
    df = add_volatility_features(df)
    df = add_technical_indicators(df)
    df = add_target_variable(df)

    df.dropna(inplace=True)  # Drop rows with NaNs introduced by rolling/lags

    df.to_csv(save_path, index=False)
    return df


In [32]:
from pathlib import Path

engineered_df = engineer_features(
    file_path="data/processed/netflix_cleaned.csv",
    save_path="data/interim/engineered_features.csv"
)
print(engineered_df.columns)
engineered_df.head()

Index(['Date', 'Close/Last', 'Volume', 'Open', 'High', 'Low', 'Year', 'Month',
       'Day', 'DayOfWeek', 'IsWeekend', 'Close/Last_lag_1', 'Close/Last_lag_2',
       'Close/Last_lag_3', 'Volume_lag_1', 'Volume_lag_2', 'Volume_lag_3',
       'Close/Last_roll_mean_7', 'Close/Last_roll_median_7',
       'Close/Last_roll_std_7', 'Close/Last_roll_max_7',
       'Close/Last_roll_min_7', 'Close/Last_roll_mean_14',
       'Close/Last_roll_median_14', 'Close/Last_roll_std_14',
       'Close/Last_roll_max_14', 'Close/Last_roll_min_14',
       'Close/Last_roll_mean_30', 'Close/Last_roll_median_30',
       'Close/Last_roll_std_30', 'Close/Last_roll_max_30',
       'Close/Last_roll_min_30', 'Volume_roll_mean_7', 'Volume_roll_median_7',
       'Volume_roll_std_7', 'Volume_roll_max_7', 'Volume_roll_min_7',
       'Volume_roll_mean_14', 'Volume_roll_median_14', 'Volume_roll_std_14',
       'Volume_roll_max_14', 'Volume_roll_min_14', 'Volume_roll_mean_30',
       'Volume_roll_median_30', 'Volume_roll_s

,Date,Close/Last,Volume,Open,High,Low,Year,Month,Day,DayOfWeek,...,Rolling_Return_30,SMA_10,EMA_10,RSI_14,MACD,MACD_Signal,Bollinger_Upper,Bollinger_Lower,Target_Close_Next_Day,Target_UpDown
30,2023-10-19,220.11,170772700.0,225.95,230.6100,216.78,2023,10,19,3,...,-1.046000,252.836,247.623987,30.809433,-3.492061,-0.583675,271.642200,230.436800,211.99,0
31,2023-10-20,211.99,138010100.0,217.01,218.8606,210.42,2023,10,20,4,...,-1.217000,247.982,241.145080,26.751966,-6.176166,-1.702173,276.341633,222.448367,212.08,1
32,2023-10-23,212.08,150683400.0,210.00,216.9800,202.51,2023,10,23,0,...,-2.050000,243.223,235.860520,28.525122,-8.201535,-3.002045,279.354604,215.944396,216.52,1
33,2023-10-24,216.52,118231100.0,216.50,222.0500,214.11,2023,10,24,1,...,-1.698667,238.513,232.344062,18.123393,-9.340710,-4.269778,280.890045,211.648955,212.42,0
34,2023-10-25,212.42,107065100.0,215.88,220.1000,212.20,2023,10,25,2,...,-1.962667,233.456,228.721505,17.381181,-10.453844,-5.506591,282.607958,207.123042,205.76,0


In [34]:
engineered_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 221 entries, 30 to 250
Data columns (total 60 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Date                       221 non-null    datetime64[ns]
 1   Close/Last                 221 non-null    float64       
 2   Volume                     221 non-null    float64       
 3   Open                       221 non-null    float64       
 4   High                       221 non-null    float64       
 5   Low                        221 non-null    float64       
 6   Year                       221 non-null    int32         
 7   Month                      221 non-null    int32         
 8   Day                        221 non-null    int32         
 9   DayOfWeek                  221 non-null    int32         
 10  IsWeekend                  221 non-null    int64         
 11  Close/Last_lag_1           221 non-null    float64       
 12  Close/Last_l